In [1]:
import json
from bs4 import BeautifulSoup as bs4
import requests
import csv
import re
MAX_DEPTH = 3
STEM = "https://en.wikipedia.org"

In [2]:
#filtering out only raw text from the html file
def visible(element):
    if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
        return False
    elif re.match('<!--.*-->', str(element.encode('utf-8'))):
        return False
    return True

In [3]:
#build the json entry for a singular page
def read_page(page):
    #if depth == -1: return
    print("reading page: " + page)
    page1 = requests.get(page)
    full_text = ""
    links=[]
    desc_links = []
    title=""
    try:
        soup = bs4(page1.text, "html5lib")
        text = soup.find_all('p')
        #extract text only before the "see also" section
        see_also = soup.find('span', id='See_also')
        title = str(soup.find('h1').getText())
        toc = soup.find("div", {"class": "toc"})
        for p in see_also.parent.previous_siblings:
            alt = p.find('img')
            if not alt and p.name == 'p':
                full_text += str(p.getText())#.encode('utf-8', 'ignore'))
                for a in p.find_all('a'):
                    if a['href'][:6] == "/wiki/": links.append(a['href'])
        for p in toc.previous_siblings:
            if p.name == "p":
                for a in p.find_all('a'):
                    if a['href'][:6] == "/wiki/": desc_links.append(a['href'])
    except AttributeError:
        print("invalid page, skipping")
    except KeyError: 
        pass
    filter(visible,full_text)
    #print(title + full_text)
    page_dict = {'title': title, 'url': page, 'links': links, 'text': full_text, 'desc_links': desc_links}
    return page_dict

In [4]:
data = {}
data['pages'] = [] #nested array so can append new pages as needed
data['pages'].append(read_page("https://en.wikipedia.org/wiki/Linear_algebra"))
origin_links = data['pages'][0]['desc_links']
for l in origin_links:
    data['pages'].append(read_page(STEM + l))

reading page: https://en.wikipedia.org/wiki/Linear_algebra
reading page: https://en.wikipedia.org/wiki/Geometry
reading page: https://en.wikipedia.org/wiki/Line_(geometry)
reading page: https://en.wikipedia.org/wiki/Plane_(geometry)
reading page: https://en.wikipedia.org/wiki/Rotation_(mathematics)
reading page: https://en.wikipedia.org/wiki/Functional_analysis
reading page: https://en.wikipedia.org/wiki/Engineering
reading page: https://en.wikipedia.org/wiki/Mathematical_model
reading page: https://en.wikipedia.org/wiki/Nonlinear_system
reading page: https://en.wikipedia.org/wiki/Matrix_(mathematics)
reading page: https://en.wikipedia.org/wiki/Vector_space
reading page: https://en.wikipedia.org/wiki/Linear_map
reading page: https://en.wikipedia.org/wiki/Mathematics
reading page: https://en.wikipedia.org/wiki/Linear_equation


In [ ]:
try: 
    data_2 = {}
    data_2['pages'] = []
    for i in range(1, len(data['pages'])):
    #for p in data['pages']:#once done, change to no include the first entry- that's our root entry and already computed
        p = data['pages'][i]
        for l in p['desc_links']:
        #print(p['url'])
            data_2['pages'].append(read_page(STEM + l))# this making it depth 3, not 2!
except ConnectionError: 
    print("wikipedia is bad, waiting 1 second")
    time.sleep(1)

reading page: https://en.wikipedia.org/wiki/Length
reading page: https://en.wikipedia.org/wiki/Area
reading page: https://en.wikipedia.org/wiki/Volume
reading page: https://en.wikipedia.org/wiki/Mathematical_science
invalid page, skipping
reading page: https://en.wikipedia.org/wiki/Axiomatic_system
reading page: https://en.wikipedia.org/wiki/Euclid
reading page: https://en.wikipedia.org/wiki/Euclid%27s_Elements
reading page: https://en.wikipedia.org/wiki/Middle_Ages
invalid page, skipping
reading page: https://en.wikipedia.org/wiki/Ren%C3%A9_Descartes
reading page: https://en.wikipedia.org/wiki/Pierre_de_Fermat
reading page: https://en.wikipedia.org/wiki/Non-Euclidean_geometry
reading page: https://en.wikipedia.org/wiki/Topological_manifold
invalid page, skipping
reading page: https://en.wikipedia.org/wiki/Ancient_Greek_language
reading page: https://en.wikipedia.org/wiki/Mathematics
reading page: https://en.wikipedia.org/wiki/Geometer
invalid page, skipping
reading page: https://en.wi

reading page: https://en.wikipedia.org/wiki/Chaos_theory
reading page: https://en.wikipedia.org/wiki/Mathematical_singularity
reading page: https://en.wikipedia.org/wiki/Randomness
reading page: https://en.wikipedia.org/wiki/Equation
reading page: https://en.wikipedia.org/wiki/Differential_equation
reading page: https://en.wikipedia.org/wiki/Polynomial
reading page: https://en.wikipedia.org/wiki/Function_(mathematics)
reading page: https://en.wikipedia.org/wiki/Linear_combination
invalid page, skipping
reading page: https://en.wikipedia.org/wiki/Variable_(mathematics)
reading page: https://en.wikipedia.org/wiki/Function_(mathematics)
reading page: https://en.wikipedia.org/wiki/Mathematics
reading page: https://en.wikipedia.org/wiki/Outline_of_physical_science
reading page: https://en.wikipedia.org/wiki/System
reading page: https://en.wikipedia.org/wiki/Proportionality_(mathematics)
reading page: https://en.wikipedia.org/wiki/Engineer
reading page: https://en.wikipedia.org/wiki/Physicis

reading page: https://en.wikipedia.org/wiki/Rotation_and_reflection_linear_transformations
reading page: https://en.wikipedia.org/wiki/Endomorphism
reading page: https://en.wikipedia.org/wiki/Linear_function
reading page: https://en.wikipedia.org/wiki/Analytic_geometry
reading page: https://en.wikipedia.org/wiki/Mathematics
reading page: https://en.wikipedia.org/wiki/Transformation_(function)
reading page: https://en.wikipedia.org/wiki/Map_(mathematics)
reading page: https://en.wikipedia.org/wiki/Module_(mathematics)
reading page: https://en.wikipedia.org/wiki/Vector_space
reading page: https://en.wikipedia.org/wiki/Scalar_(mathematics)
reading page: https://en.wikipedia.org/wiki/Real_number
reading page: https://en.wikipedia.org/wiki/Complex_number
reading page: https://en.wikipedia.org/wiki/Coefficient
invalid page, skipping
reading page: https://en.wikipedia.org/wiki/Field_(mathematics)
invalid page, skipping
reading page: https://en.wikipedia.org/wiki/Mathematics
reading page: http

In [ ]:
with open('linalg_desclinks.json', 'w') as f: 
    json.dump(data, f, indent=4, sortKeys = True)
    json.dump(data_2, f, indent=4, sortKeys=True)